Initialize Sentiment Analyzer

In [ ]:
import nltk
import warnings
warnings.filterwarnings('ignore')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

Updating Vader Lexicon with External Lexicon

In [ ]:
import csv
import pandas as pd

# stock market lexicon
stock_lex = pd.read_csv('lexicon_data/stock_lex.csv')
stock_lex['sentiment'] = (stock_lex['Aff_Score'] + stock_lex['Neg_Score'])/2
stock_lex = dict(zip(stock_lex.Item, stock_lex.sentiment))
stock_lex = {k:v for k,v in stock_lex.items() if len(k.split(' '))==1}
stock_lex_scaled = {}
for k, v in stock_lex.items():
    if v > 0:
        stock_lex_scaled[k] = v / max(stock_lex.values()) * 4
    else:
        stock_lex_scaled[k] = v / min(stock_lex.values()) * -4

# Loughran and McDonald
positive = []
with open('lexicon_data/lm_positive.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        positive.append(row[0].strip())
    
negative = []
with open('lexicon_data/lm_negative.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        entry = row[0].strip().split(" ")
        if len(entry) > 1:
            negative.extend(entry)
        else:
            negative.append(entry[0])

final_lex = {}
final_lex.update({word:2.0 for word in positive})
final_lex.update({word:-2.0 for word in negative})
final_lex.update(stock_lex_scaled)
final_lex.update(sia.lexicon)
sia.lexicon = final_lex

In [ ]:
# spider headlines and generate sentiment score from "nasdaq.com"

from selenium import webdriver
import pandas as pd
import time
from params import *


website = 'http://www.nasdaq.com/symbol/MSFT/news-headlines'
def getText(someList):
    returnList = []
    for i in someList:
        returnList.append(i.text)
    return returnList

dr = webdriver.Chrome('./chromedriver')
dr.get(website)
titles=[]
dates=[]
scores =[]
next = dr.find_elements_by_class_name('pagination__next')

while(next[0].is_enabled()):
    texts = getText(dr.find_elements_by_class_name('quote-news-headlines__item-title'))
    date = getText(dr.find_elements_by_class_name('quote-news-headlines__date'))
    for t,d in zip (texts,date):
        titles.append(t)
        dates.append(d)
        score = sia.polarity_scores(t)['compound']
        scores.append(score)
    dr.execute_script("arguments[0].click();", next[0])
    time.sleep(2)

table = pd.DataFrame([titles,dates,scores])
table.to_csv("Headlines.csv")

In [ ]:
import numpy as np
a= np.array([dates,scores]).T
table = pd.DataFrame(a,columns=['date','score'])
table['score'] = pd.to_numeric(table['score'])
a = table.groupby(['date']).mean()
a.to_csv('MSFT.csv')


In [ ]:
from selenium import webdriver
import pandas as pd
import time
from params import *
from bs4 import BeautifulSoup

dr = webdriver.Chrome('./chromedriver')
#website = 'https://www.businesstimes.com.sg/search/facebook?page=#main-content'
for i in range(1,50):
    website = 'https://www.businesstimes.com.sg/search/facebook?page='+str(i)
    dr.get(website)
    time.sleep(2)
    dr.refresh()

    page = dr.page_source
    soup = BeautifulSoup(page, features="html.parser")
    posts = soup.findAll("div", {"class": "media-body"})
    date_sentiments = {}
    for post in posts:
        time.sleep(1)
        url = post.a['href']
        date = post.time.text
        print(date, url)
        try:
            #link_page = urlopen(url).read()
            dr.get(url)
        except:
            url = url[:-2]
            #link_page = urlopen(url).read()
            dr.get(url)
        dr.refresh()
        link_page = dr.page_source
        link_soup = BeautifulSoup(link_page)
        sentences = link_soup.findAll("p")
        passage = ""
        for sentence in sentences:
            passage += sentence.text
        sentiment = sia.polarity_scores(passage)['compound']
        print(sentiment)
        date_sentiments.setdefault(date, []).append(sentiment)

date_sentiment = {}

for k,v in date_sentiments.items():
    date_sentiment[datetime.strptime(k, '%d %b %Y').date() + timedelta(days=1)] = round(sum(v)/float(len(v)),3)

#earliest_date = min(date_sentiment.keys())

print(date_sentiment)

In [ ]:
date_sentiment = {}
from datetime import datetime
from datetime import timedelta
for k,v in date_sentiments.items():
    date_sentiment[datetime.strptime(k, '%d %b %Y').date() + timedelta(days=1)] = round(sum(v)/float(len(v)),3)

#earliest_date = min(date_sentiment.keys())

print(date_sentiment)

In [ ]:
date_sentiments